# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [2]:
poly_modulus = 64
bit_sizes= [40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[0, 0, 1099511623296, 0, 0, 1, 0, 1099511623296, 1, 1, 1099511623296, 1, 1, 1, 1099511623296, 1, 1099511623296, 1, 0, 0, 1099511623296, 1, 1, 1099511623296, 1099511623296, 0, 0, 0, 0, 0, 0, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1099511623296, 1099511623296, 0, 1, 1, 1099511623296, 0, 1, 0, 1099511623296, 1, 1099511623296, 1]]


In [4]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [5]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[1]     [0, 0, 0, 1]     [0, 1]


### Decodes back to Integer

In [6]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

1
8
2


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [7]:
encrypter = Encryptor(ctx, pk)

In [8]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [9]:
decrypter = Decryptor(ctx, sk)

In [10]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [11]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

1     8     2


## Evaluator

In [12]:
eval = Evaluator(ctx)

In [13]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

9


In [14]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

9


In [15]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

9


### Verify result

In [16]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [17]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[127467458230, 644314413317, 204785670438, 750837692148, 136960509837, 1093509326191, 860783479643, 756743342323, 405027555463, 1083275239148, 205019059481, 225199556806, 925805718786, 499720023403, 371200675131, 1038781305241, 759012902516, 1059705199397, 61017040272, 625861354154, 570148665491, 34912981542, 429409110715, 909006816393, 585787448234, 854159639079, 264326846874, 820396173723, 20007182420, 844360972333, 1076987137653, 884329380575, 959853615141, 458930278236, 42810442255, 808585438264, 759464852937, 365284216606, 781954897265, 383488359137, 708111279244, 594709524499, 628635894240, 585911086356, 726931939379, 594996700726, 817845186417, 395369077259, 239232021256, 1063103518356, 419374985589, 1080283371425, 449079163667, 548081908262, 759412338624, 616945580445, 1011370468696, 856408221719, 155089635110, 1092979658621, 161110519292, 51135746339, 945967841325, 933499164988]], [[133818496119, 943075325841, 56580002692, 758168077802, 1055602744683, 943669089615, 2

In [18]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

8      8


## Tensor Testing

In [19]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
context = Context(EncryptionParams(128, CoeffModulus().create(64, [30, 30]), 64))

In [20]:
key_generator = KeyGenerator(context)
keys = key_generator.keygen()
relin_key = key_generator.get_relin_keys()

In [21]:
x = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [22]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12d856520>

In [23]:
y = x.decrypt(private_key = keys[0])
y

tensor([[1, 2, 3],
        [4, 5, 6]])

In [24]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12d856520>

In [25]:
x = x + y

In [26]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12dbe41f0>

In [27]:
x.decrypt(private_key = keys[0])

tensor([[ 2,  4,  6],
        [ 8, 10, 12]])

In [28]:
x = x - th.Tensor([[1,2,3],[1,2,3]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [29]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12dc0e1f0>

In [30]:
x.decrypt(private_key = keys[0])

tensor([[1, 2, 3],
        [7, 8, 9]])

In [31]:
 x = x*x

In [32]:
x.decrypt(private_key = keys[0])

tensor([[ 1,  4,  9],
        [49, 64, 81]])

In [33]:
a = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)
b = th.Tensor([[-1,-2],[-3,-4], [-5,-6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [34]:
a

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12dc202e0>

In [35]:
b

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12dc20e50>

In [36]:
ans_mm = a.mm(b, relin_key = relin_key)
print(ans_mm)

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12d808400>


In [37]:
mm_decrypt = ans_mm.decrypt(private_key = keys[0])

In [38]:
print(mm_decrypt)

tensor([[-22, -28],
        [-49, -64]])
